<a href="https://colab.research.google.com/github/Erasnilson/Reg-Log-XGBoots-AWS/blob/main/Xgboost_classificacao_census_comentado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressão Logística via algoritmo XGBoots - Aplicação via Sagemaker da AWS (Modelagem Machine Learn)


## **Tratamento da base de dados**


### - Organizar a base de dados no formato {y, x1, x2, ..., xn} para modelagem de classificação **via Sagemaker da AWS**.
```
colunas = []
colunas.append('income')                        # incluíndo a primeira coluna var resposta y (necessário no AWS)
for i in range(len(base_census.columns[:-1])):  # removendo a última coluna y - icome
     colunas.append(base_census.columns[i])
base_census = base_census[colunas]
base_census
```

### - Configurar a **variável resposta como dummy**.
- Se y for ' >50K'-> 1, caso contrário ->0 (AWS só compila valor
numérico)

```
def functiondymmy(text):
    if text == ' >50K':
        return 1.0
    else:
        return 0.0
```

- aplicar a transformação dicotómica:
```
base_census['income'] = base_census['income'].apply(functiondymmy)
```

### - As variáveis qualitativas não são convertidas automaticamente em dummys(matriz de delineamento), logo, os fatores (variáveis qualitativas) como sexo, educação etc, precisam ser ser **transformadas em números** para a análise do algoritmo.
```
base_census = pd.get_dummies(base_census,prefix=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'inative-country'],
                            columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race',
                            'sex', 'inative-country'],
                            dtype='int') #incluido dtype='int', pois no colab não identificou 0 ou 1
```

### - As especificações das Variáveis X e Y, via AWS, são realizadas **sem os rótulos das variáveis e sem os índices**. Assim, o algoritmo deverá compilar apenas os valores que serão transformados em formato binário.
```
X_teste = base_teste.iloc[:,1:len(base_census)].values
y_teste = base_teste.iloc[:, 0].values
```


# **Configurações SageMaker**

## - A configuração do Sagemaker envolve a **comunicação do AWS(sagemaker)** com python e o **acesso para a gravação de arquivos** específicos (treino e teste)

```
import sagemaker
import boto3                           # faz a comunicação da AWS com o python (específica da AWS)
from sagemaker import Session
import sagemaker.amazon.common as smac # sagemaker commom library
import io                              # grava a base de dados no formato S3
import os                              # utilizado para acessar os recursos do sistema operacional
```

## - Após a criação do bucket (cursoawssagemaker) e do usuário, é necessário especificar os caminhos (diretórios) para a geração dos modelos e para os arquivos de treino e teste, além de configurar as permissões adequadas.

```
session = sagemaker.Session()
bucket = 'cursoawssagemaker'               # bucket armazenado no S3
subpasta_modelo = 'modelos/census/xgboost' # local para armazenar os modelos
subpasta_dataset = 'datasets/census'
key_train = 'census-train-data-xgboost'    # nome do arquivo train
key_test = 'census-test-data-xgboost'      # nome do arquivo teste
role = sagemaker.get_execution_role()      # permissões

# variável que indica o caminho para o arquivo train e test
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)                                 
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)

# local onde o modelo será armazenado no S3
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))

# print para visualização dos caminhos especificados
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))
```

## - Pro fim, é necessário converter os dados que estão em formato numpy para binário, para enviar ao S3.

```
with open('census_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

with open('census_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)
```



# **Treinamento do XGBoost**

## Ao realizar o treinamento do modelo é preciso espcificar uma instância, a depender da instância selecionada pode ser cobrado um valor por hora. Para mais detalhes, consultar a tabela de preço do sagemaker https://aws.amazon.com/pt/sagemaker/pricing/.



```
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')
```


### - Blocos de anotações do Studio e instâncias de blocos de anotações sob demanda - Uso do nível **gratuito** por mês **pelos primeiros dois meses** - **250 horas da instância ml.t3.medium** em blocos de anotações do Studio OU **250 horas da instância ml.t2 medium** ou **ml.t3.medium em blocos de anotações sob demanda**.

🔴 A instância **ml.m5.2xlarge** (tem 8vCPU e 32 GiB) **custa	0,461 USD/h**, conforme custos https://aws.amazon.com/pt/sagemaker/pricing/
```
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
xgboost = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)
```




```
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgboost.set_hyperparameters(num_round = 100, objective = 'reg:logistic')   #trabalhando com reg logística
```




```
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
data_channels = {'train': train_input, 'validation': validation_input}
```



```
xgboost.fit(data_channels) #realizando o ajuste do modelo
```

Ao final, verificar ser o Endpoint foi gerado no ``inference'' do sagemaker.
🔴 OBS, após utilizar o **Endpoint** apagar para não haver cobranças.




# **Deploy, previsões e avaliação**

- Avaliação do Modelo

```
xgboost_classifier = xgboost.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')
```

- configurar o classificador, agora serão convertidas para csv (antes estava em numpy)

```
from sagemaker.serializers import CSVSerializer
xgboost_classifier.serializer = CSVSerializer()
X_teste.shape, type(X_teste)
```


- Realizar as previsões. Agora, são retornadas as probabilidades (0 - 1). Uma alternativa é reclassificar, se >= 0,5 será classificado como 1, caso contrário 0.

```
# Não converter true=1, o python deverá ajustar automaticamente
previsoes = (previsoes >= 0.5)
print(previsoes)
```

- Utilizar o pacote sklearn para obter matrix de confusão do modelo.

```
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_teste, previsoes)
cm
```

- No exemplo do curso, nota-se que a acurácia do modelo foi XGboost foi melhor que o linear-learner. Abaixo temos, que o algoritmo XGboost apresentou um desempenho de recall de 0,66 para a classificação 1, com uma precisão de 78% e para a classificação 0, tem-se um recall de 94% com precisão de 89%.

```
print(classification_report(y_teste, previsoes))
              precision    recall  f1-score   support

         0.0       0.89      0.94      0.92      7365
         1.0       0.78      0.66      0.71      2404

    accuracy                           0.87      9769
   macro avg       0.84      0.80      0.81      9769
weighted avg       0.87      0.87      0.87      9769
```






# **Tuning**

- No tuning iremos gerar vários modelos a partir de reamostragens, nesse caso serão solicitado **9 modelo**. Deve-se utilizar a comparação do **menor erro** para obter as estimativas dos parâmetros e realizar o novo modelo com tais estimativas.

```
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha"
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "max_depth"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "300",
          "MinValue": "50",
          "Name": "num_round"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 9,        #<-------------------------
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:error",    #<--e não rmse---------------
      "Type": "Minimize"
    }
  }
```




```
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": container,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_train_data
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_test_data
          }
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,subpasta_modelo)
    },
    "ResourceConfig": {
      "InstanceCount": 2,
      "InstanceType": "ml.c4.2xlarge",
      "VolumeSizeInGB": 10
    },
    "RoleArn": role,
    "StaticHyperParameters": {
      "eval_metric": "error",
      "objective": "binary:logistic",
      "rate_drop": "0.3",
      "tweedie_variance_power": "1.4"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}
```


- Por fim, deve-se executar o codigo a seguir para realizar o tunning dos parâmetros. Ao finalizar, pode-se verificar os modelos e suas estatísticas na guia Training, Hyperparameter tuning jobs (clicar no best training job) - copiar os parâmetros e jogar no novo modelo.

```
smclient = boto3.client('sagemaker')
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = "xgboosttuningcensus",
                                          HyperParameterTuningJobConfig = tuning_job_config,
                                          TrainingJobDefinition = training_job_definition)

```


# **Construção do novo modelo**


- Ao verificar o melhor modelo, obtido pelo tuning, deve-se copiar as estimativas dos parâmetros e substituir no parâmetros da programação a seguir:

```
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='latest')
xgboost_tuning = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)
xgboost_tuning.set_hyperparameters(num_round = 102, eta = 0.14507612435685635,
                                   min_child_weight = 2.412681801757289,
                                   alpha = 0.3189676727624047, tweedie_variance_power = 1.4,
                                   rate_drop = 0.3)
xgboost_tuning.fit(data_channels)
```

- Geração dos Endpoints

```
xgboost_classifier_tuning = xgboost_tuning.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')
```

- Buscar as previsões (probabilidades)

```
from sagemaker.serializers import CSVSerializer
xgboost_classifier_tuning.serializer = CSVSerializer()
previsoes = np.array(xgboost_classifier_tuning.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)
```

- Transformar as previsões

```
previsoes = (previsoes >= 0.5)
# deixar em formato inteiro (saida em true or false)
y_teste = np.array(y_teste).astype(int)
y_teste   # previsoes.shape, y_teste.shape
```
- Obtendo acurácia do modelo

```
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_teste, previsoes)
cm
#accuracy_score(y_teste, previsoes)
```
- Resultado final

```
print(classification_report(y_teste, previsoes))
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      7365
           1       0.80      0.64      0.71      2404

    accuracy                           0.87      9769
   macro avg       0.84      0.80      0.82      9769
weighted avg       0.87      0.87      0.87      9769
```



## A seguir, será apresentado os comando a serem processados na AWS. Destaco que este notebook foi adaptado de uma atividade de curso, resolvi compartilhar sem compilar no Aws, pois não possuo a contra free.

In [1]:
!pip install pandas
!pip install numpy


In [2]:
import pandas as pd
import numpy as np


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
# Importanto data frame - Colab
base_census = pd.read_csv('/content/drive/MyDrive/Github/Aws-classificacao-Xboost/census.csv')
#base_census.head(5)

In [49]:
#base_census = pd.read_csv('census.csv') # forma simples de carregar no AWS
# migrando a variável y para a primeira coluna - necessário no Sagemaker Aws
colunas = []
colunas.append('income')                        # incluíndo primeiro name y
for i in range(len(base_census.columns[:-1])):  # removendo a última coluna y - icome
     colunas.append(base_census.columns[i])
#base_census = base_census[colunas]
#base_census
colunas

['income',
 'age',
 'workclass',
 'final-weight',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loos',
 'hour-per-week',
 'inative-country']

In [50]:
#base_census = pd.read_csv('census.csv') # forma simples de carregar no AWS
# migrando a variável y para a primeira coluna - necessário no Sagemaker Aws
colunas = []
colunas.append('income')                        # incluíndo primeiro name y
for i in range(len(base_census.columns[:-1])):  # removendo a última coluna y - icome
     colunas.append(base_census.columns[i])
base_census = base_census[colunas]
base_census.head(5)


,income,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,inative-country
0,<=50K,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,<=50K,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,<=50K,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,<=50K,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,<=50K,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [51]:
def functiondymmy(text):
    if text == ' >50K':
        return 1.0
    else:
        return 0.0

In [52]:
base_census['income'] = base_census['income'].apply(functiondymmy)

In [53]:
base_census.head(5)

,income,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,inative-country
0,0.0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,0.0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,0.0,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,0.0,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,0.0,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [54]:
base_census = pd.get_dummies(base_census,prefix=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'inative-country'],
                            columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'inative-country'],
                            dtype='int') #incluido dtype='int', pois no colab não identificou 0 ou 1
base_census.head(5)

,income,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,...,inative-country_ Portugal,inative-country_ Puerto-Rico,inative-country_ Scotland,inative-country_ South,inative-country_ Taiwan,inative-country_ Thailand,inative-country_ Trinadad&Tobago,inative-country_ United-States,inative-country_ Vietnam,inative-country_ Yugoslavia
0,0.0,39,77516,13,2174,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.0,50,83311,13,0,0,13,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.0,38,215646,9,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.0,53,234721,7,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.0,28,338409,13,0,0,40,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
base_treinamento = base_census.iloc[0:22792,:]
base_treinamento.shape

(22792, 109)

In [56]:
base_teste = base_census.iloc[22792:,:]
base_teste.shape

(9769, 109)

In [57]:
22792 + 9769

32561

## Especificação das Variáveis X e Y (AWS é necessário remover os rótulos das variáveis)

In [58]:
X_teste = base_teste.iloc[:,1:len(base_census)].values
X_teste

array([[    30,  75167,     13, ...,      1,      0,      0],
       [    39, 176296,      9, ...,      1,      0,      0],
       [    19,  93518,     10, ...,      1,      0,      0],
       ...,
       [    58, 151910,      9, ...,      1,      0,      0],
       [    22, 201490,      9, ...,      1,      0,      0],
       [    52, 287927,      9, ...,      1,      0,      0]])

In [59]:
X_teste.shape

(9769, 108)

In [60]:
y_teste = base_teste.iloc[:, 0].values
y_teste

array([0., 1., 0., ..., 0., 0., 1.])

In [ ]:
base_treinamento.to_csv('census_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('census_test_xgboost.csv', header = False, index = False)

# Configurações SageMaker

In [ ]:
import sagemaker
import boto3
from sagemaker import Session
import sagemaker.amazon.common as smac # sagemaker commom library
import io
import os

In [ ]:
session = sagemaker.Session()
bucket = 'cursoawssagemaker'
subpasta_modelo = 'modelos/census/xgboost'
subpasta_dataset = 'datasets/census'
key_train = 'census-train-data-xgboost'
key_test = 'census-test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

Role: arn:aws:iam::936535973187:role/service-role/AmazonSageMaker-ExecutionRole-20220510T125992
Localização da base de treinamento: s3://cursoawssagemaker/datasets/census/train/census-train-data-xgboost
Localização da base de teste: s3://cursoawssagemaker/datasets/census/test/census-test-data-xgboost
Modelo final será salvo em: s3://cursoawssagemaker/modelos/census/xgboost/output


In [ ]:
with open('census_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [ ]:
with open('census_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

# Treinamento do XGBoost

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')

In [ ]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
xgboost = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgboost.set_hyperparameters(num_round = 100, objective = 'reg:logistic')

In [ ]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
data_channels = {'train': train_input, 'validation': validation_input}

In [ ]:
xgboost.fit(data_channels)

2022-05-17 15:52:00 Starting - Starting the training job...
2022-05-17 15:52:29 Starting - Preparing the instances for trainingProfilerReport-1652802720: InProgress
.........
2022-05-17 15:53:57 Downloading - Downloading input data...
2022-05-17 15:54:17 Training - Downloading the training image...
2022-05-17 15:54:57 Training - Training image download completed. Training in progress..Arguments: train
[2022-05-17:15:54:56:INFO] Running standalone xgboost training.
[2022-05-17:15:54:56:INFO] File size need to be processed in the node: 7.07mb. Available memory size in the node: 23504.35mb
[2022-05-17:15:54:56:INFO] Determined delimiter of CSV input is ','
[15:54:56] S3DistributionType set as FullyReplicated
[15:54:56] 22792x108 matrix with 2461536 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-05-17:15:54:56:INFO] Determined delimiter of CSV input is ','
[15:54:56] S3DistributionType set as FullyReplicated
[15:54:56] 9769x108 matrix with 1055052 

# Deploy, previsões e avaliação

In [ ]:
xgboost_classifier = xgboost.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

-------!

In [ ]:
from sagemaker.serializers import CSVSerializer
xgboost_classifier.serializer = CSVSerializer()

In [ ]:
X_teste.shape, type(X_teste)

((9769, 108), numpy.ndarray)

In [ ]:
previsoes = np.array(xgboost_classifier.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)
previsoes

array([7.1665001e-01, 9.8000973e-01, 8.9295841e-05, ..., 1.3065693e-02,
       2.0877716e-04, 9.9958044e-01], dtype=float32)

In [ ]:
previsoes = (previsoes >= 0.5)
print(previsoes)

[ True  True False ... False False  True]


In [ ]:
previsoes.shape, y_teste.shape

((9769,), (9769,))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
cm = confusion_matrix(y_teste, previsoes)
cm

array([[6914,  451],
       [ 821, 1583]])

In [ ]:
accuracy_score(y_teste, previsoes)

0.8697921998157436

In [ ]:
print(classification_report(y_teste, previsoes))

              precision    recall  f1-score   support

         0.0       0.89      0.94      0.92      7365
         1.0       0.78      0.66      0.71      2404

    accuracy                           0.87      9769
   macro avg       0.84      0.80      0.81      9769
weighted avg       0.87      0.87      0.87      9769



# Tuning

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha"
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "max_depth"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "300",
          "MinValue": "50",
          "Name": "num_round"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 9,
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:error",
      "Type": "Minimize"
    }
  }

In [ ]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": container,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_train_data
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_test_data
          }
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,subpasta_modelo)
    },
    "ResourceConfig": {
      "InstanceCount": 2,
      "InstanceType": "ml.c4.2xlarge",
      "VolumeSizeInGB": 10
    },
    "RoleArn": role,
    "StaticHyperParameters": {
      "eval_metric": "error",
      "objective": "binary:logistic",
      "rate_drop": "0.3",
      "tweedie_variance_power": "1.4"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}

In [ ]:
smclient = boto3.client('sagemaker')
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = "xgboosttuningcensus",
                                          HyperParameterTuningJobConfig = tuning_job_config,
                                          TrainingJobDefinition = training_job_definition)

# Construção do novo modelo

In [ ]:
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='latest')
xgboost_tuning = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)
xgboost_tuning.set_hyperparameters(num_round = 102, eta = 0.14507612435685635,
                                   min_child_weight = 2.412681801757289,
                                   alpha = 0.3189676727624047, tweedie_variance_power = 1.4,
                                   rate_drop = 0.3)
xgboost_tuning.fit(data_channels)

2022-05-17 16:06:12 Starting - Starting the training job...ProfilerReport-1652803572: InProgress
...
2022-05-17 16:06:52 Starting - Preparing the instances for training......
2022-05-17 16:08:11 Downloading - Downloading input data...
2022-05-17 16:08:29 Training - Downloading the training image.....Arguments: train
[2022-05-17:16:09:20:INFO] Running standalone xgboost training.
[2022-05-17:16:09:20:INFO] File size need to be processed in the node: 7.07mb. Available memory size in the node: 23867.78mb
[2022-05-17:16:09:20:INFO] Determined delimiter of CSV input is ','
[16:09:20] S3DistributionType set as FullyReplicated
[16:09:20] 22792x108 matrix with 2461536 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-05-17:16:09:20:INFO] Determined delimiter of CSV input is ','
[16:09:20] S3DistributionType set as FullyReplicated
[16:09:20] 9769x108 matrix with 1055052 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,

In [ ]:
xgboost_classifier_tuning = xgboost_tuning.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

------!

In [ ]:
from sagemaker.serializers import CSVSerializer
xgboost_classifier_tuning.serializer = CSVSerializer()
previsoes = np.array(xgboost_classifier_tuning.predict(X_teste).decode('utf-8').split(',')).astype(np.float32)

In [ ]:
previsoes

array([ 0.690067  ,  0.91854674, -0.00379324, ...,  0.01779461,
       -0.00613862,  0.9809268 ], dtype=float32)

In [ ]:
previsoes = (previsoes >= 0.5)
previsoes

array([ True,  True, False, ..., False, False,  True])

In [ ]:
y_teste = np.array(y_teste).astype(int)
y_teste

array([0, 1, 0, ..., 0, 0, 1])

In [ ]:
previsoes.shape, y_teste.shape

((9769,), (9769,))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_teste, previsoes)
cm

array([[6974,  391],
       [ 855, 1549]])

In [ ]:
accuracy_score(y_teste, previsoes)

0.8724536800081891

In [ ]:
print(classification_report(y_teste, previsoes))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      7365
           1       0.80      0.64      0.71      2404

    accuracy                           0.87      9769
   macro avg       0.84      0.80      0.82      9769
weighted avg       0.87      0.87      0.87      9769

